# Project 3 - Wrangle OpenStreetMap New York Metropolitan Area

Christian Guzman

June 2017

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict

In [ ]:
#Main OSM
OSM_FILE = "C:/Users/cguzm_000/Desktop/Desk Docs/Data/newyork.osm"
#k = 500

In [ ]:
osm_file = "C:/Users/cguzm_000/Documents/udacity-projects/data/sample.osm"

In [2]:
def count_tags(file):
    unique_tags = defaultdict(int)
    for event, elem in ET.iterparse(file, events=("start",)):
        unique_tags[elem.tag] += 1
    return unique_tags

In [3]:
count_tags(osm_file)

defaultdict(int,
            {'member': 174,
             'nd': 29795,
             'node': 22975,
             'osm': 1,
             'relation': 19,
             'tag': 19426,
             'way': 3599})

In [17]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        ######
        if lower.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["lower_colon"] += 1
        elif problemchars.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["problemchars"] += 1
        else:
            #print(element.attrib['k'])
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [18]:
process_map(osm_file)

{'lower': 7525, 'lower_colon': 11664, 'other': 189, 'problemchars': 48}

In [19]:
#EXPLORING USERS
def get_user(element):
    return element.attrib.get('uid')


def process_users(filename):
    users = [] #set()
    for _, elem in ET.iterparse(filename, events=("start",)):
        if get_user(elem):
            users.append(get_user(elem))
    users = set(users)
        
    return users

In [20]:
process_users(osm_file)

{'100643',
 '1012362',
 '102691',
 '103253',
 '104519',
 '104962',
 '1051550',
 '105609',
 '105839',
 '105946',
 '1072240',
 '1087647',
 '109276',
 '1093326',
 '109362',
 '1110270',
 '111159',
 '113450',
 '115611',
 '11617',
 '1168086',
 '118134',
 '1185712',
 '118871',
 '119748',
 '119881',
 '120146',
 '1202134',
 '121241',
 '1219875',
 '1221143',
 '1221149',
 '1221152',
 '12290',
 '1231593',
 '1232894',
 '123364',
 '123633',
 '1240849',
 '1240864',
 '12448',
 '125718',
 '12581',
 '126442',
 '1289164',
 '129614',
 '130472',
 '1306',
 '1311287',
 '13203',
 '1323363',
 '1323730',
 '132803',
 '1337142',
 '134892',
 '135329',
 '1357992',
 '1376118',
 '1380816',
 '1384038',
 '13944',
 '139856',
 '1408522',
 '141082',
 '1424765',
 '1426068',
 '14293',
 '143002',
 '146680',
 '147510',
 '147851',
 '148169',
 '1484939',
 '14850',
 '148800',
 '1494110',
 '150272',
 '150368',
 '150692',
 '1520225',
 '152074',
 '153669',
 '154730',
 '155262',
 '158826',
 '1597155',
 '160042',
 '160949',
 '161619'

In [21]:
#IMPROVING STREET NAMES
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place",
            "Square", "Lane", "Road", "Trail", "Parkway", "Commons"]
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [28]:
pprint.pprint(dict(audit(osm_file)))

{'10': set(['Route 10']),
 '27': set(['NJ 27']),
 'A': set(['Avenue A']),
 'Ave': set(['Park Ave']),
 'B': set(['Avenue B']),
 'Broadway': set(['Broadway', 'East Broadway', 'West Broadway']),
 'Circle': set(['64th Circle', 'Alex Circle', 'Covington Circle']),
 'Concourse': set(['Grand Concourse']),
 'Crescent': set(['246th Crescent',
                  'Asquith Crescent',
                  'Burden Crescent',
                  'Cromwell Crescent']),
 'E': set(['Bay Drive E', 'Lakeview Avenue E', 'Locust Avenue E']),
 'East': set(['Belmar Drive East',
              'Clearview Expressway Service Road East',
              'Drumgoole Road East',
              'Park Drive East']),
 'Extension': set(['Eastern Parkway Extension']),
 'Green': set(['Carlyle Green', 'Forest Green']),
 'Heights': set(['Columbia Heights']),
 'Highway': set(['Kings Highway', 'Montauk Highway']),
 'Hl': set(['Diana Hl']),
 'K': set(['Avenue K']),
 'L': set(['Avenue L']),
 'Loop': set(['Bethel Loop',
              'Mar

In [44]:
mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "N": "North",
            "S": "South",
            "E": "East",
            "W": "West",
            "Hl": "Hill",
            "Ter": "Terrace",
            "Pkwy": "Parkway",
            "Sr": "State Road"
           }

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m and m.group() in mapping.keys():
        name = street_type_re.sub(mapping[m.group()], name)
    return name

In [40]:
def test():
    for st_type, ways in audit(osm_file).iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            if name is not better_name:
                print name, "=>", better_name
test()

Diana Hl => Diana Hill
Lido Pkwy => Lido Parkway
Granada Pkwy => Granada Parkway
Locust Avenue E => Locust Avenue East
Lakeview Avenue E => Lakeview Avenue East
Bay Drive E => Bay Drive East
Spur Drive N => Spur Drive North
Salisbury Drive N => Salisbury Drive North
Avenue N => Avenue North
Horace Harding Expressway Sr S => Horace Harding Expressway Sr South
Avenue S => Avenue South
Parkway Drive S => Parkway Drive South
Somerset Road W => Somerset Road West
Bayview Avenue W => Bayview Avenue West
Avenue W => Avenue West
Park Ave => Park Avenue
Grand St => Grand Street
McKinley Ter => McKinley Terrace


## Updating "Sr" to "State Road"

Some of the adresses contained the abbreviation **Sr** which stands for **State Road**.

In [56]:
sr_re = re.compile(r'\bSr\b')
def update_name(name, mapping):
    m = street_type_re.search(name)
    state_road = sr_re.search(tag.attrib['v'])
    if m and m.group() in mapping.keys():
        name = street_type_re.sub(mapping[m.group()], name)
    if state_road:
        name = sr_re.sub(mapping[state_road.group()], name)
    return name


for element in get_element(osm_file, tags=('node','way')):
    for tag in element.iter("tag"):
        state_road = sr_re.search(tag.attrib['v'])
        if state_road:
            print update_name(tag.attrib['v'], mapping)
            

Horace Harding Expressway State Road South
Grand Central Parkway State Road South
Horace Harding Expressway State Road South


In [23]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version',
               'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset',
              'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def shape_element(element, node_attr_fields=NODE_FIELDS,
                  way_attr_fields=WAY_FIELDS, problem_chars=PROBLEMCHARS,
                  default_tag_type='regular'):
    '''Clean and shape node or way XML element to python dict'''

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements
    # MY CODE HERE
    if element.tag == 'node':
        for k in element.attrib:
            if k in node_attr_fields:
                node_attribs[k] = element.attrib[k]
        if len(element.findall('tag')) > 0:
            for tag in element.iter('tag'):
                problem = PROBLEMCHARS.search(tag.attrib['k'])
                if problem:
                    print(problem.group())
                    continue
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                tag_dict['value'] = tag.attrib['v']
                # Dealing with colons
                m = LOWER_COLON.search(tag.attrib['k'])
                if m:
                    key_split = tag.attrib['k'].split(":")
                    key = ':'.join(key_split[1:])
                    tag_dict['key'] = key
                    tag_dict['type'] = key_split[0]
                else:
                    tag_dict['key'] = tag.attrib['k']
                    tag_dict['type'] = default_tag_type
                tags.append(tag_dict)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for k in element.attrib:
            if k in way_attr_fields:
                way_attribs[k] = element.attrib[k]
        if len(element.findall('tag')) > 0:
            for tag in element.iter('tag'):
                problem = PROBLEMCHARS.search(tag.attrib['k'])
                if problem:
                    print(problem.group())
                    continue
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                # Dealing with colons
                m = LOWER_COLON.search(tag.attrib['k'])
                if m:
                    key_split = tag.attrib['k'].split(":")
                    key = ':'.join(key_split[1:])
                    tag_dict['key'] = key
                    tag_dict['type'] = key_split[0]
                else:
                    tag_dict['key'] = tag.attrib['k']
                    tag_dict['type'] = default_tag_type
                tag_dict['value'] = tag.attrib['v']    
                
                tags.append(tag_dict)
        if len(element.findall('nd')) > 0:
            position = 0
            for nd in element.iter('nd'):
                node_dict = {}
                node_dict['id'] = element.attrib['id']
                node_dict['node_id'] = nd.attrib['ref']
                node_dict['position'] = position
                way_nodes.append(node_dict)
                position += 1
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

def test():
    for element in get_element(osm_file, tags=('node', 'way')):
        el = shape_element(element)